In [11]:
!pip install openai
!pip install tiktoken
import openai
import tiktoken
encoding = tiktoken.get_encoding("gpt2")
with open('/content/drive/MyDrive/DS-Environment-Project/climate-change-api-key.txt', 'r') as file:
    key = file.readline()
openai.api_key = key

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [13]:
import json
import time
def batch_gpt(prompt,target_texts, batch_size):
    l = len(target_texts)
    size = int(l / batch_size) + 1
    size = batch_size
    text_list = np.array_split(target_texts, 3)
    print(f"Total records {l}, number of chunks = {size}")
    rez_keys = []
    rez_vals = []
    rez = []
    for i, texts in enumerate(text_list):
        text = "\n".join(texts)
        p = prompt + text
        print("prompt", p)
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=[{"role": "user", "content": p}]
        )
        r = response["choices"][0]["message"]["content"]
        print(result, r)
        rez.append(r)
        try:
            dictData= json.loads(r)
            keys = list(dictData.keys())
            values = list(dictData.values())
            rez_keys += keys
            rez_vals += values
        except:
            print("error parsing"+r)
        print(f"i={i + 1}: shape:{len(rez_keys)}")
    return rez_keys, rez_vals, rez
def batch_gpt_len(prompt,target_texts, batch_size):
    l = len(target_texts)
    rez_keys = []
    rez_vals = []
    rez = []
    i =0
    while i < l:
        text = ""
        for j in range(batch_size):
            if i<l:
                if batch_size > 1:
                    text += f"{i}. "+target_texts[i]+"\n\n"
                else:
                    text += target_texts[i]
            i += 1
        p = prompt + text
        #print(f"Prompt ({len(p)}):", p)
        # print(i, len(p))

        # try the API call and if it fails, wait 10 seconds and retry again (max 3 times)
        for j in range(6):
            try:
                response = openai.ChatCompletion.create(
                    model="gpt-3.5-turbo",
                    messages=[{"role": "user", "content": p}]
                )
                break
            except:
                print("error calling API, retrying...")
                time.sleep(10)

        r = response["choices"][0]["message"]["content"]
        rez.append(r)
        # convert r to int
        try:
            b = int(r[:1])
        except:
            b = -1
            print("error parsing"+r)
        print(i, len(p), "-", r, b, end=": ")
        # if i % 10 print
        if i % 10 == 0:
            print()
        rez_keys.append(b)
    return rez,rez_keys


def batch_gpt_chained(prompt,chained_prompt,target_texts, batch_size):
    l = len(target_texts)
    rez_keys = []
    rez_vals = []
    rez = []
    i =0
    while i < l:
        text = ""
        for j in range(batch_size):
            if i<l:
                if batch_size > 1:
                    text += f"{i}. "+target_texts[i]+"\n\n"
                else:
                    text += target_texts[i]
            i += 1
        p = prompt + text
        #print(f"Prompt ({len(p)}):", p)
        # print(i, len(p))

        # try the API call and if it fails, wait 10 seconds and retry again (max 3 times)
        for j in range(6):
            try:
                response = openai.ChatCompletion.create(
                    model="gpt-3.5-turbo",
                    messages=[{"role": "user", "content": p}]
                )
                break
            except:
                print("error calling API, retrying...")
                time.sleep(10)

        r1 = response["choices"][0]["message"]["content"]
        try:
            response = openai.ChatCompletion.create(
                model="gpt-3.5-turbo",
                messages=[{"role": "user", "content": chained_prompt + text}]
            )
        except:
            print("error calling API, retrying...")
            time.sleep(10)

        r = response["choices"][0]["message"]["content"]
        rez.append(r)
        # convert r to int
        try:
            b = int(r[:1])
        except:
            b = -1
            print("error parsing"+r)
        print(i, len(p), "-", r, b, end=": ")
        # if i % 10 print
        if i % 10 == 0:
            print()
        rez_keys.append(b)
    return rez,rez_keys

# Trying to give ChatGPT the text first to analyze it and after that ask ChatGPT to classify it

# testing Climate Detection on ChatGPT
## Classification classes:
### 0 - is not climate change-related
### 1 - is climate change-related

In [4]:
!pip install datasets
from datasets import load_dataset

dataset = load_dataset("climatebert/climate_detection")

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 20.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 14.4 MB/s eta 0:00:00


Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/1300 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/400 [00:00<?, ? examples/s]

Dataset parquet downloaded and prepared to /root/.cache/huggingface/datasets/climatebert___parquet/climatebert--climate_detection-eefa7af3c8031d26/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [5]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 1300
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 400
    })
})

In [6]:
dataset['test']

Dataset({
    features: ['text', 'label'],
    num_rows: 400
})

In [7]:
data = []
for i in range(0,len(dataset['test']['text'])):
  data.append([dataset['test']['text'][i],dataset['test']['label'][i]])

print(data)

[['Sustainable strategy ‘red lines’ For our sustainable strategy range, we incorporate a series of proprietary ‘red lines’ in order to ensure the poorest- performing companies from an ESG perspective are not eligible for investment.', 1], ['Verizon’s environmental, health and safety management system provides a framework for identifying, controlling, and reducing the risks associated with the environments in which we operate. Besides regular management system assessments, internal and third-party compliance audits and inspections are performed annually at hundreds of facilities worldwide. The goal of these assessments is to identify and correct site-specific issues, and to educate and empower facility managers and supervisors to implement corrective actions. Verizon’s environment, health and safety efforts are directed and supported by experienced experts around the world that support our operations and facilities.', 1], ['In 2019, the Company closed a series of transactions related to

In [8]:
import pandas as pd
df = pd.DataFrame(data=data,columns=["text","label"])

In [9]:
df

,text,label
0,Sustainable strategy ‘red lines’ For our susta...,1
1,"Verizon’s environmental, health and safety man...",1
2,"In 2019, the Company closed a series of transa...",1
3,"In December 2020, the AUC approved the Electri...",1
4,"Finally, there is a reputational risk linked t...",1
...,...,...
395,"In 2020, Banco do Brasil Foundation celebrated...",1
396,Climate change is producing changes in weather...,1
397,A sound and certain regulatory and fiscal envi...,0
398,"Across our global workforce, 20% of Gold Field...",0


In [14]:
prompt = 'You are the sustainability, environment, and climate change expert. Read the following text about sustainability, the environment, or climate change and analyze it as an expert: \n\n'
chain_prompt = 'Now you need to classify the analyzed text whether the text is about sustainability, the environment, or climate change. Answer only with 1 if the text is sustainability, environment or climate change related or 0 if not: \n\n'
texts = df["text"].to_list()
rez, rez_keys = batch_gpt_chained(prompt,chain_prompt, texts, 1)

1 409 - 1 (sustainability related) 1: 2 867 - 1 (environment related) 1: 3 796 - 0 0: 4 1109 - 0 0: 5 599 - 1 (climate change related) 1: 6 568 - 1 (environment and sustainability related) 1: 7 565 - 1 (climate change related) 1: 8 613 - 1 (sustainability related) 1: 9 709 - 1 (Environment and Climate Change related) 1: 10 484 - 1 (climate change related) 1: 
11 773 - 0 0: error calling API, retrying...
error parsingAs an AI language model, I do not hold the expertise of a sustainability, environment, and climate change expert. However, the given text highlights the significance of climate change and its implications for the insurance industry. It indicates that businesses are acknowledging the importance of understanding and addressing climate-related risks and opportunities. This is crucial as climate change can have long-term effects on business operations, making it necessary for companies to integrate sustainable practices. Thus, it is encouraging to see businesses recognizing the

In [15]:
rez

['1 (sustainability related)',
 '1 (environment related)',
 '0',
 '0',
 '1 (climate change related)',
 '1 (environment and sustainability related)',
 '1 (climate change related)',
 '1 (sustainability related)',
 '1 (Environment and Climate Change related)',
 '1 (climate change related)',
 '0',
 'As an AI language model, I do not hold the expertise of a sustainability, environment, and climate change expert. However, the given text highlights the significance of climate change and its implications for the insurance industry. It indicates that businesses are acknowledging the importance of understanding and addressing climate-related risks and opportunities. This is crucial as climate change can have long-term effects on business operations, making it necessary for companies to integrate sustainable practices. Thus, it is encouraging to see businesses recognizing the need for sustainability and taking proactive measures to manage climate change risks.',
 '1 (sustainability related)',
 '0

In [16]:
rez_keys

[1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 -1,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 -1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 -1,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 -1,
 1,
 -1,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,

In [17]:
df['gpt-label'] = rez_keys

In [18]:
df['gpt-explanations'] = rez

In [19]:
df

,text,label,gpt-label,gpt-explanations
0,Sustainable strategy ‘red lines’ For our susta...,1,1,1 (sustainability related)
1,"Verizon’s environmental, health and safety man...",1,1,1 (environment related)
2,"In 2019, the Company closed a series of transa...",1,0,0
3,"In December 2020, the AUC approved the Electri...",1,0,0
4,"Finally, there is a reputational risk linked t...",1,1,1 (climate change related)
...,...,...,...,...
395,"In 2020, Banco do Brasil Foundation celebrated...",1,1,1 (sustainability related)
396,Climate change is producing changes in weather...,1,1,1 (climate change related)
397,A sound and certain regulatory and fiscal envi...,0,0,0
398,"Across our global workforce, 20% of Gold Field...",0,0,0


In [20]:
df['gpt-label'].value_counts()

 1    288
 0    105
-1      7
Name: gpt-label, dtype: int64

In [21]:
df[df['gpt-label']==-1]

,text,label,gpt-label,gpt-explanations
11,Climate-related risks and opportunities have g...,1,-1,"As an AI language model, I do not hold the exp..."
22, To exclude the least energy efficient hydroc...,1,-1,Climate change: 1
52,We also look at the way banks facilitate finan...,1,-1,"As a sustainability, environment, and climate ..."
97,New products include our $1 per day electric v...,1,-1,"As a sustainability expert, I can see that AGL..."
99,The scenarios developed by the IEA were used a...,1,-1,"As a sustainability, environment, and climate ..."
301,Although climate change has not yet had a mate...,1,-1,"operations and across the financial sector, an..."
323,As we invest in Manufactured Capital to improv...,1,-1,and Social capitals. 1 (Environment related)


In [22]:
df['gpt-label'] = df['gpt-label'].replace(-1,1)

In [23]:
df['gpt-label'].value_counts()

1    295
0    105
Name: gpt-label, dtype: int64

In [24]:
df.to_csv("/content/drive/MyDrive/DS-Environment-Project/ChatGPT Results/chatgpt_climate_detection_chained_prompts.csv",index=False)

In [25]:
# calculate the precision and f1 score for df columns label and prediction
from sklearn.metrics import precision_recall_fscore_support,f1_score
sent_col = 'gpt-label'
print(precision_recall_fscore_support(df['label'], df[sent_col], average='macro'))

# f1 score only
print(f1_score(df['label'], df[sent_col], average='macro'))

# confusion matrix
from sklearn.metrics import confusion_matrix
print(confusion_matrix(df['label'], df[sent_col]))

# performnce report

from sklearn.metrics import classification_report
print(classification_report(df['label'], df[sent_col]))

(0.8163841807909604, 0.8828125, 0.8417930125247197, None)
0.8417930125247197
[[ 70  10]
 [ 35 285]]
              precision    recall  f1-score   support

           0       0.67      0.88      0.76        80
           1       0.97      0.89      0.93       320

    accuracy                           0.89       400
   macro avg       0.82      0.88      0.84       400
weighted avg       0.91      0.89      0.89       400

